In [9]:
import os
import json
from PIL import Image, ImageDraw
import pandas as pd
import torch
import numpy as np

In [10]:
anlables_path = '../../../labels'
alltest_path = '../../../test/Images/'
result_path = '../../../detection-results'

In [11]:
# 读取txt文件
def getFileList(path):
    for root, dirs, files in os.walk(path):
        return files
anfile_names = getFileList(anlables_path)
anfile_names

['01_0001.txt',
 '01_0006.txt',
 '01_0007.txt',
 '01_0009.txt',
 '01_0010.txt',
 '01_0011.txt',
 '01_0015.txt',
 '01_0017.txt',
 '01_0018.txt',
 '01_0020.txt',
 '02_0001.txt',
 '02_0002.txt',
 '02_0003.txt',
 '02_0006.txt',
 '02_0007.txt',
 '02_0008.txt',
 '02_0009.txt',
 '02_0011.txt',
 '02_0014.txt',
 '02_0019.txt',
 '03_0001.txt',
 '03_0004.txt',
 '03_0007.txt',
 '03_0008.txt',
 '03_0013.txt',
 '03_0014.txt',
 '03_0018.txt',
 '03_0019.txt',
 '03_0020.txt']

In [12]:
# 读取测试所有文件
def getTestList(path):
    for root, dirs, files in os.walk(path):
        return files
file_test_lists = getFileList(alltest_path)
file_test_lists

['01_0001.jpg',
 '01_0002.jpg',
 '01_0003.jpg',
 '01_0004.jpg',
 '01_0005.jpg',
 '01_0006.jpg',
 '01_0007.jpg',
 '01_0008.jpg',
 '01_0009.jpg',
 '01_0010.jpg',
 '01_0011.jpg',
 '01_0012.jpg',
 '01_0013.jpg',
 '01_0014.jpg',
 '01_0015.jpg',
 '01_0016.jpg',
 '01_0017.jpg',
 '01_0018.jpg',
 '01_0019.jpg',
 '01_0020.jpg',
 '02_0001.PNG',
 '02_0002.PNG',
 '02_0003.PNG',
 '02_0004.PNG',
 '02_0005.PNG',
 '02_0006.PNG',
 '02_0007.PNG',
 '02_0008.PNG',
 '02_0009.PNG',
 '02_0010.PNG',
 '02_0011.PNG',
 '02_0012.PNG',
 '02_0013.PNG',
 '02_0014.PNG',
 '02_0015.PNG',
 '02_0016.PNG',
 '02_0017.PNG',
 '02_0018.PNG',
 '02_0019.PNG',
 '02_0020.PNG',
 '03_0001.PNG',
 '03_0002.PNG',
 '03_0003.PNG',
 '03_0004.PNG',
 '03_0005.PNG',
 '03_0006.PNG',
 '03_0007.PNG',
 '03_0008.PNG',
 '03_0009.PNG',
 '03_0010.PNG',
 '03_0011.PNG',
 '03_0012.PNG',
 '03_0013.PNG',
 '03_0014.PNG',
 '03_0015.PNG',
 '03_0016.PNG',
 '03_0017.PNG',
 '03_0018.PNG',
 '03_0019.PNG',
 '03_0020.PNG']

In [13]:
# 获取 image-size
def getImageSize(path,filename,type):
    image = Image.open(os.path.join(path,filename + type))
    return image.size
getImageSize(alltest_path,os.path.splitext(anfile_names[0])[0],'.jpg')

(1182, 816)

In [14]:
# box [cx,cy,w,h]
def convert(size, box):
     # 解宽高归一化
    x_center = box[0] * size[0]
    y_center = box[1] * size[1]

    xmin = (2 * x_center - box[2] * size[0]) / 2.0
    xmax = (2 * x_center + box[2] * size[0]) / 2.0
    
    ymin = (2 * y_center - box[3] * size[1]) / 2.0
    ymax = (2 * y_center + box[3] * size[1]) / 2.0

#     xmin = int(((float(box[0]))*size[0]+1)-(float(box[2]))*0.5*size[0])
#     ymin = int(((float(box[1]))*size[1]+1)-(float(box[3]))*0.5*size[1])

#     xmax = int(((float(box[0]))*size[0]+1)+(float(box[2]))*0.5*size[0])
#     ymax = int(((float(box[1]))*size[1]+1)+(float(box[3]))*0.5*size[1])

    return (int(xmin), int(ymin), int(xmax), int(ymax))

In [15]:
# 生成结果文件
def pro_result(file_test_lists,file_name,file_path,test_path,result_path=result_path):
    '''
        file_test_lists: 所有测试图片的名字
        file_name: 从yolo检测到异常的label
        file_path: 异常图片的label的path位置
        file_test: 所有测试图片的图片的path位置
        result_path: 输出文件位置
    '''
    for fileitem in file_test_lists:
        testfile = os.path.splitext(fileitem)[0]
        type = os.path.splitext(fileitem)[1]
        print(testfile)
        if testfile+'.txt' in file_name:
            abnormfile = testfile + '.txt'
            # 读取txt文件
            with open(os.path.join(file_path,abnormfile),'r') as load_f:
                for item in load_f:
                    # txt内容
                    item_args = item.split()
                    img_w, img_h = getImageSize(test_path,os.path.splitext(abnormfile)[0],type)
                    box = (float(item_args[1]), float(item_args[2]),
                        float(item_args[3]), float(item_args[4]))

                    # 转换为 box [xmin,ymin,xmax,ymax]
                    boxex = convert((img_w, img_h), box)

                    # 写到一个新的文件里面
                    with open(os.path.join(result_path,abnormfile), 'a') as file:
                        file.write("defect" + " " + str(round(float(item_args[5]), 2)) + " " + " ".join([str(s) for s in boxex]) + '\n')
                        # file.write(item_args[0] + " " + " ".join([str(s) for s in box]) + '\n')
        else:
            # 正常图片直接给空
            emptyfile = testfile + '.txt'
            with open(os.path.join(result_path,emptyfile), 'w') as file:
                pass


In [16]:
pro_result(file_test_lists,anfile_names,anlables_path,alltest_path)

01_0001
01_0002
01_0003
01_0004
01_0005
01_0006
01_0007
01_0008
01_0009
01_0010
01_0011
01_0012
01_0013
01_0014
01_0015
01_0016
01_0017
01_0018
01_0019
01_0020
02_0001
02_0002
02_0003
02_0004
02_0005
02_0006
02_0007
02_0008
02_0009
02_0010
02_0011
02_0012
02_0013
02_0014
02_0015
02_0016
02_0017
02_0018
02_0019
02_0020
03_0001
03_0002
03_0003
03_0004
03_0005
03_0006
03_0007
03_0008
03_0009
03_0010
03_0011
03_0012
03_0013
03_0014
03_0015
03_0016
03_0017
03_0018
03_0019
03_0020
